In [1]:
# Import necessary Libraries
import numpy as np
import pandas as pd
import sys  
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 10, 8
from pprint import pprint

In [2]:
# Load Data
foreign_mat = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Foreign_Matter.csv')
micro = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Micro_Screen.csv')
moisture = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Moisture_Content.csv')
solvent = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Solvent.csv')
potency = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/potencytests.csv') 
samples = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Sample_lab_time.csv') 

In [3]:
# Create Test Type Column
foreign_mat['Test'] = 'Foreign Matter'
micro['Test'] = 'Micro Screen'
moisture['Test'] = 'Moisture Content'
solvent['Test'] = 'Solvent Screen'

# Combine Foreign Matter, Moisture Content and Solvent Screening
tests = pd.concat([foreign_mat, micro, moisture, solvent])
tests.sort_values('sample_id', ascending=True, inplace=True )
tests.head()

,sample_id,name,value,failure,location,orgid,Test
374,19,yeast_and_mold,8800.0,0,139,170,Micro Screen
373,19,bile_tolerant,0.0,0,139,170,Micro Screen
372,19,aerobic_bacteria,0.0,0,139,170,Micro Screen
74,19,moisture,6.0,0,139,170,Moisture Content
148,19,Stems,0.0,0,139,170,Foreign Matter


In [4]:
# Subset Potency data to lab and location level only
labs = pd.DataFrame(samples, columns=['sample_id', 'test_date', 'lab_license'])
labs.sort_values('sample_id', ascending=True, inplace=True )
labs.head()

,sample_id,test_date,lab_license
0,18,6/19/2014,4
1,19,6/19/2014,4
2,20,6/19/2014,4
3,21,6/19/2014,4
4,22,6/19/2014,4


In [5]:
merge = pd.merge(tests, labs, on='sample_id', how='left')
merge.head()

,sample_id,name,value,failure,location,orgid,Test,test_date,lab_license
0,19,yeast_and_mold,8800.0,0,139,170,Micro Screen,6/19/2014,4
1,19,bile_tolerant,0.0,0,139,170,Micro Screen,6/19/2014,4
2,19,aerobic_bacteria,0.0,0,139,170,Micro Screen,6/19/2014,4
3,19,moisture,6.0,0,139,170,Moisture Content,6/19/2014,4
4,19,Stems,0.0,0,139,170,Foreign Matter,6/19/2014,4


In [8]:
fails = merge.groupby(['location', 'Test'])['failure'].sum().reset_index()
#Lab_group = spanish.sort_values('spanish', ascending=False)
fails.head()

,location,Test,failure
0,124,Foreign Matter,0
1,124,Micro Screen,31
2,124,Moisture Content,1
3,126,Foreign Matter,0
4,126,Micro Screen,4


In [9]:
test_tot = merge.groupby(['location', 'Test'])['failure'].count().reset_index()
#Lab_group = spanish.sort_values('spanish', ascending=False)
test_tot.head()

,location,Test,failure
0,124,Foreign Matter,260
1,124,Micro Screen,665
2,124,Moisture Content,130
3,126,Foreign Matter,474
4,126,Micro Screen,1545


In [12]:
fail_merge = pd.merge(fails, test_tot, on=['location','Test'], how='left')
fail_merge = fail_merge.rename(columns = {
    'failure_x':'Num Fail', 'failure_y':'Total Tests'})
fail_merge['Fail Rate'] = fail_merge['Num Fail']/fail_merge['Total Tests']
fail_merge.head()

,location,Test,Num Fail,Total Tests,Fail Rate
0,124,Foreign Matter,0,260,0.000000
1,124,Micro Screen,31,665,0.046617
2,124,Moisture Content,1,130,0.007692
3,126,Foreign Matter,0,474,0.000000
4,126,Micro Screen,4,1545,0.002589


In [13]:
Labs_pivot = merge.pivot_table(values=['failure'], index=['lab_license', 'Test'], aggfunc=np.sum)
Labs_pivot

failure
lab_license Test                     
1           Foreign Matter          0
            Micro Screen          148
            Moisture Content        4
            Solvent Screen          0
2           Foreign Matter         33
            Micro Screen          344
            Moisture Content        9
            Solvent Screen          3
3           Foreign Matter          8
            Micro Screen         3448
            Moisture Content       34
            Solvent Screen        175
4           Foreign Matter          0
            Micro Screen         4537
            Moisture Content      186
            Solvent Screen        244
5           Foreign Matter          0
            Micro Screen           55
            Moisture Content        0
            Solvent Screen          6
6           Foreign Matter          1
            Micro Screen          476
            Moisture Content       83
            Solvent Screen          1
7           Foreign Matter          0
            Micro Screen          103
            Moisture Content       14
            Solvent Screen          0
8           Foreign Matter          2
            Micro Screen          290
...                               ...
12          Micro Screen         1311
            Moisture Content       27
            Solvent Screen         42
13          Foreign Matter          0
            Micro Screen          299
            Moisture Content       10
            Solvent Screen          0
14          Foreign Matter          3
            Micro Screen          589
            Moisture Content        9
            Solvent Screen          0
15          Foreign Matter          0
            Micro Screen          235
            Moisture Content        1
            Solvent Screen          2
16          Foreign Matter          1
            Micro Screen          133
            Moisture Content        1
            Solvent Screen          0
17          Foreign Matter          0
            Micro Screen          174
            Moisture Content        2
            Solvent Screen          7
18          Foreign Matter          0
            Micro Screen           37
            Moisture Content        0
            Solvent Screen          0
123456      Foreign Matter          0
            Micro Screen           10
            Moisture Content        0

[75 rows x 1 columns]

In [17]:
Labs_pivot_count = merge.pivot_table(values=['failure'], index=['lab_license', 'Test'], aggfunc=[np.sum, len])
Labs_pivot_count

sum     len
                             failure failure
lab_license Test                            
1           Foreign Matter         0     764
            Micro Screen         148    3810
            Moisture Content       4     378
            Solvent Screen         0      25
2           Foreign Matter        33    1460
            Micro Screen         344    9877
            Moisture Content       9     724
            Solvent Screen         3    1250
3           Foreign Matter         8   66484
            Micro Screen        3448  189285
            Moisture Content      34   32948
            Solvent Screen       175    3142
4           Foreign Matter         0   18298
            Micro Screen        4537   62251
            Moisture Content     186    9083
            Solvent Screen       244    2538
5           Foreign Matter         0    1712
            Micro Screen          55    4815
            Moisture Content       0     851
            Solvent Screen         6      92
6           Foreign Matter         1    5246
            Micro Screen         476   17032
            Moisture Content      83    2620
            Solvent Screen         1      64
7           Foreign Matter         0   24342
            Micro Screen         103   68570
            Moisture Content      14   12071
            Solvent Screen         0     862
8           Foreign Matter         2    5938
            Micro Screen         290   18365
...                              ...     ...
12          Micro Screen        1311   76130
            Moisture Content      27   13863
            Solvent Screen        42     887
13          Foreign Matter         0    1312
            Micro Screen         299    3500
            Moisture Content      10     656
            Solvent Screen         0       2
14          Foreign Matter         3    2278
            Micro Screen         589    7880
            Moisture Content       9    1138
            Solvent Screen         0     351
15          Foreign Matter         0   21052
            Micro Screen         235   57620
            Moisture Content       1   10526
            Solvent Screen         2     886
16          Foreign Matter         1     820
            Micro Screen         133    2280
            Moisture Content       1     410
            Solvent Screen         0      25
17          Foreign Matter         0     712
            Micro Screen         174    1940
            Moisture Content       2     356
            Solvent Screen         7      28
18          Foreign Matter         0     196
            Micro Screen          37     630
            Moisture Content       0      98
            Solvent Screen         0      12
123456      Foreign Matter         0       8
            Micro Screen          10      20
            Moisture Content       0       4

[75 rows x 2 columns]

In [7]:
fails = tests.groupby(['Test'])['failure'].sum().reset_index()
#Lab_group = spanish.sort_values('spanish', ascending=False)
fails.head(25)

,Test,failure
0,Foreign Matter,121
1,Micro Screen,14400
2,Moisture Content,441
3,Solvent Screen,561


In [8]:
tests['failure'].value_counts()

0    921465
1     15523
Name: failure, dtype: int64

In [13]:
# Create a set of dummy variables from the sex variable
df_all = pd.get_dummies(tests['Test'])
# Join the dummy variables to the main dataframe
tests_new = pd.concat([tests, df_all], axis=1)
tests_new.head(20)

,sample_id,name,value,failure,location,orgid,Test,Foreign Matter,Micro Screen,Moisture Content,Solvent Screen
374,19,yeast_and_mold,8800.0,0,139,170,Micro Screen,0.0,1.0,0.0,0.0
373,19,bile_tolerant,0.0,0,139,170,Micro Screen,0.0,1.0,0.0,0.0
372,19,aerobic_bacteria,0.0,0,139,170,Micro Screen,0.0,1.0,0.0,0.0
74,19,moisture,6.0,0,139,170,Moisture Content,0.0,0.0,1.0,0.0
148,19,Stems,0.0,0,139,170,Foreign Matter,1.0,0.0,0.0,0.0
371,19,coliforms,0.0,0,139,170,Micro Screen,0.0,1.0,0.0,0.0
370,19,e_coli_and_salmonella,0.0,0,139,170,Micro Screen,0.0,1.0,0.0,0.0
149,19,Other,0.0,0,139,170,Foreign Matter,1.0,0.0,0.0,0.0
377,20,aerobic_bacteria,200.0,0,139,170,Micro Screen,0.0,1.0,0.0,0.0
379,20,coliforms,0.0,0,139,170,Micro Screen,0.0,1.0,0.0,0.0


In [14]:
total_tests = pd.DataFrame(merge, columns=['location', 'Foreign Matter', 'Micro Screen','Moisture Content', 'Solvent Screen'])
total_tests.head()

,location,Foreign Matter,Micro Screen,Moisture Content,Solvent Screen
0,139,NaN,NaN,NaN,NaN
1,139,NaN,NaN,NaN,NaN
2,139,NaN,NaN,NaN,NaN
3,139,NaN,NaN,NaN,NaN
4,139,NaN,NaN,NaN,NaN


In [38]:
Lab_group = total_tests.groupby(['lab_license'])['Foreign Matter', 'Micro Screen', 'Moisture Content', 'Solvent Screen'].sum().reset_index()
#Lab_group = spanish.sort_values('spanish', ascending=False)
Lab_group.head(25)

,lab_license,Foreign Matter,Micro Screen,Moisture Content,Solvent Screen
0,1,764.0,3810.0,378.0,25.0
1,2,1460.0,9877.0,724.0,1250.0
2,3,66484.0,189285.0,32948.0,3142.0
3,4,18298.0,62251.0,9083.0,2538.0
4,5,1712.0,4815.0,851.0,92.0
5,6,5246.0,17032.0,2620.0,64.0
6,7,24342.0,68570.0,12071.0,862.0
7,8,5938.0,18365.0,2949.0,113.0
8,9,20944.0,72140.0,10443.0,2871.0
9,10,5286.0,14605.0,2628.0,298.0


In [41]:
fails2 = pd.DataFrame(merge, columns=['lab_license', 'Foreign Matter', 'Micro Screen','Moisture Content', 'Solvent Screen', 'failure'])
fails2.head()

,lab_license,Foreign Matter,Micro Screen,Moisture Content,Solvent Screen,failure
0,4,0.0,1.0,0.0,0.0,0
1,4,0.0,1.0,0.0,0.0,0
2,4,0.0,1.0,0.0,0.0,0
3,4,0.0,0.0,1.0,0.0,0
4,4,1.0,0.0,0.0,0.0,0


In [15]:
Labs_pivot2 = merge.pivot_table(values=['failure'], index=['location'], columns=['Test'], aggfunc=[np.sum,len])
Labs_pivot2

sum                                               \
                failure                                                
Test     Foreign Matter Micro Screen Moisture Content Solvent Screen   
location                                                               
124                 0.0         31.0              1.0            NaN   
126                 0.0          4.0              1.0            4.0   
127                 0.0         35.0              0.0            NaN   
139                 0.0         19.0              0.0            0.0   
140                 0.0         21.0              0.0            NaN   
141                 0.0         51.0              0.0            NaN   
142                 0.0         17.0              0.0            NaN   
143                 0.0          1.0              0.0            NaN   
144                 0.0         16.0              0.0            NaN   
145                 0.0         19.0              0.0            NaN   
146                 1.0          8.0              1.0            NaN   
147                 0.0        128.0              2.0            NaN   
148                 0.0         12.0              0.0            0.0   
149                 0.0         33.0              0.0            NaN   
150                 0.0         28.0              4.0            NaN   
151                 0.0          0.0              0.0            NaN   
152                 2.0        210.0              2.0            NaN   
153                 1.0        156.0              5.0            0.0   
154                 0.0          1.0              1.0            NaN   
155                 4.0         70.0              0.0            NaN   
156                 0.0          8.0              0.0            NaN   
157                 0.0          3.0              0.0            NaN   
158                 0.0          1.0              0.0            NaN   
159                 0.0         16.0              2.0            NaN   
160                 0.0          7.0              0.0            NaN   
161                 0.0          5.0              0.0            0.0   
162                 0.0         74.0              6.0            NaN   
163                 0.0         33.0              0.0            NaN   
165                 0.0          1.0              0.0            NaN   
166                 0.0         73.0              0.0            NaN   
...                 ...          ...              ...            ...   
1931                0.0          0.0              0.0            NaN   
1933                0.0          0.0              0.0            NaN   
1941                0.0          0.0              0.0            NaN   
1946                0.0          1.0              0.0            NaN   
1949                0.0          0.0              0.0            NaN   
1953                0.0          5.0              0.0            NaN   
1954                0.0          0.0              0.0            NaN   
1987                0.0          0.0              0.0            NaN   
1988                0.0          3.0              0.0            NaN   
2002                0.0          0.0              1.0            NaN   
2004                0.0          0.0              0.0            NaN   
2010                0.0          0.0              0.0            NaN   
2017                NaN          0.0              NaN            NaN   
2021                0.0          0.0              0.0            NaN   
2023                0.0          3.0              0.0            1.0   
2025                0.0          0.0              0.0            NaN   
2049                NaN          0.0              NaN            NaN   
2050                NaN          0.0              NaN            NaN   
2058                0.0          0.0              0.0            NaN   
2065                NaN          NaN              NaN            0.0   
2102                NaN          0.0              NaN       

In [17]:
Labs_pivot2.to_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Producer_fail_rate.csv', index=True)

In [8]:
# Subset tests data for merging
tests = pd.DataFrame(tests, columns=['sample_id', 'failure','Test'])
tests.sort_values('sample_id', ascending=True, inplace=True )
tests.head()

,sample_id,failure,Test
374,19,0,Micro Screen
373,19,0,Micro Screen
372,19,0,Micro Screen
74,19,0,Moisture Content
148,19,0,Foreign Matter


In [35]:
Total_tests = pd.DataFrame(merge, columns=['lab_license', 'Test'])
# Create a set of dummy variables from the sex variable
tests_all = pd.get_dummies(tests['Test'])
# Join the dummy variables to the main dataframe
tests_new = pd.concat([tests, df_all], axis=1)
tests_new.head()
Total_tests.head()

,lab_license,Test
0,4,Micro Screen
1,4,Micro Screen
2,4,Micro Screen
3,4,Moisture Content
4,4,Foreign Matter


In [6]:
#Define a generic function using Pandas replace function
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

tests["failure"] = coding(tests["failure"], {0:'Pass',1:'Fail'})


tests_new['Foreign Matter'] = coding(tests_new['Foreign Matter'], {0.0:'Pass',1.0:'Fail'})
tests_new['Micro Screen'] = coding(tests_new['Micro Screen'], {0.0:'Pass',1.0:'Fail'})
tests_new['Moisture Content'] = coding(tests_new['Moisture Content'], {0.0:'Pass',1.0:'Fail'})
tests_new['Solvent Screen'] = coding(tests_new['Solvent Screen'], {0.0:'Pass',1.0:'Fail'})
tests_new.head(3)
tests.head()

,sample_id,name,value,failure,location,orgid,Test
374,19,yeast_and_mold,8800.0,Pass,139,170,Micro Screen
373,19,bile_tolerant,0.0,Pass,139,170,Micro Screen
372,19,aerobic_bacteria,0.0,Pass,139,170,Micro Screen
74,19,moisture,6.0,Pass,139,170,Moisture Content
148,19,Stems,0.0,Pass,139,170,Foreign Matter


In [14]:
# Subset tests data for merging
tests_samples = pd.DataFrame(tests, columns=['orgid', 'failure', 'location','Test'])
tests_samples.sort_values('orgid', ascending=True, inplace=True )
tests_samples.head()

,orgid,failure,location,Test
39976,163,0,142,Moisture Content
5034,163,0,142,Foreign Matter
5035,163,0,142,Foreign Matter
36588,163,0,142,Moisture Content
36587,163,0,142,Moisture Content


In [13]:
# Subset Potency data to lab and location level only
labs = pd.DataFrame(samples, columns=['sample_id', 'test_date', 'lab_license'])
labs.sort_values('sample_id', ascending=True, inplace=True )
labs.head()

,sample_id,test_date,lab_license
0,18,6/19/2014,4
1,19,6/19/2014,4
2,20,6/19/2014,4
3,21,6/19/2014,4
4,22,6/19/2014,4


In [19]:
merge = pd.merge(tests, labs, on='sample_id', how='left')
merge.head(7)

,sample_id,name,value,failure,location,orgid,Test,test_date,lab_license
0,19,yeast_and_mold,8800.0,0,139,170,Micro Screen,6/19/2014,4
1,19,bile_tolerant,0.0,0,139,170,Micro Screen,6/19/2014,4
2,19,aerobic_bacteria,0.0,0,139,170,Micro Screen,6/19/2014,4
3,19,moisture,6.0,0,139,170,Moisture Content,6/19/2014,4
4,19,Stems,0.0,0,139,170,Foreign Matter,6/19/2014,4
5,19,coliforms,0.0,0,139,170,Micro Screen,6/19/2014,4
6,19,e_coli_and_salmonella,0.0,0,139,170,Micro Screen,6/19/2014,4


In [20]:
# impute_grps = data.pivot_table(values=["LoanAmount"], index=["Gender","Married","Self_Employed"], aggfunc=np.mean)

Labs_pivot = merge.pivot_table(values=['failure'], index=['lab_license', 'Test'], aggfunc=np.sum)
Labs_pivot


failure
lab_license Test                     
1           Foreign Matter          0
            Micro Screen          148
            Moisture Content        4
            Solvent Screen          0
2           Foreign Matter         33
            Micro Screen          344
            Moisture Content        9
            Solvent Screen          3
3           Foreign Matter          8
            Micro Screen         3448
            Moisture Content       34
            Solvent Screen        175
4           Foreign Matter          0
            Micro Screen         4537
            Moisture Content      186
            Solvent Screen        244
5           Foreign Matter          0
            Micro Screen           55
            Moisture Content        0
            Solvent Screen          6
6           Foreign Matter          1
            Micro Screen          476
            Moisture Content       83
            Solvent Screen          1
7           Foreign Matter          0
            Micro Screen          103
            Moisture Content       14
            Solvent Screen          0
8           Foreign Matter          2
            Micro Screen          290
...                               ...
12          Micro Screen         1311
            Moisture Content       27
            Solvent Screen         42
13          Foreign Matter          0
            Micro Screen          299
            Moisture Content       10
            Solvent Screen          0
14          Foreign Matter          3
            Micro Screen          589
            Moisture Content        9
            Solvent Screen          0
15          Foreign Matter          0
            Micro Screen          235
            Moisture Content        1
            Solvent Screen          2
16          Foreign Matter          1
            Micro Screen          133
            Moisture Content        1
            Solvent Screen          0
17          Foreign Matter          0
            Micro Screen          174
            Moisture Content        2
            Solvent Screen          7
18          Foreign Matter          0
            Micro Screen           37
            Moisture Content        0
            Solvent Screen          0
123456      Foreign Matter          0
            Micro Screen           10
            Moisture Content        0

[75 rows x 1 columns]

In [18]:
print(len(merge))

936988


In [7]:
samples.head()

,id,inventoryid,quantity,sessiontime,inventorytype,strain,product_name,deleted,result,lab_license,...,transactionid,parentid,received,received_quantity,transactionid_original,sample_amount_used,sample_amount_destroyed,sample_amount_other,other_sample_id,inventoryparentid
0,18,3.972540e+14,7.0,1403191624,13,Double Purple Doja,NaN,1,0,4,...,34955,6.032850e+15,0,NaN,34955,NaN,NaN,NaN,NaN,6.032850e+15
1,19,5.721620e+15,7.0,1403191980,13,Dutch Hawaiian,NaN,0,1,4,...,34977,6.032850e+15,1,7.0,34977,7.0,NaN,NaN,NaN,6.032850e+15
2,20,6.843060e+14,7.0,1403192191,13,Shiska Berry,NaN,0,1,4,...,34990,6.032850e+15,1,7.0,34990,7.0,NaN,NaN,NaN,6.032850e+15
3,21,8.933330e+15,7.0,1403192572,13,Sleestack,NaN,0,1,4,...,35012,6.032850e+15,1,7.0,35012,7.0,NaN,NaN,NaN,6.032850e+15
4,22,2.103310e+15,7.0,1403192711,13,Space Needle,NaN,0,1,4,...,35019,6.032850e+15,1,7.0,35019,7.0,NaN,NaN,NaN,6.032850e+15


In [11]:
# Subset Potency data to lab and location level only
labs_samples = pd.DataFrame(samples, columns=['orgid', 'lab_license'])
labs_samples.sort_values('orgid', ascending=True, inplace=True )
labs_samples.head(20)

,orgid,lab_license
61420,163,7
71520,163,7
71519,163,7
71518,163,7
71517,163,7
71516,163,7
55399,163,7
55400,163,7
55401,163,7
55403,163,7


In [ ]:
merge = pd.merge(tests_samples, labs, on='location', how='left')
merge.head(20)